# Edgar Filings - Financials

This notebook shows how to use Algoseek's Edgar Filings dataset to extract financial statements from 10-K and 10-Q filings. The notebook shows how to extract a list of all available financial statements for a given company and how to extract the content of a specific financial statement.

To start, we need to create a connection to the Algoseek Edgar Filings database. The service can be accessed using the `clickhouse_connect` library. The connection parameters are provided by Algoseek.


In [3]:
import os
import pandas as pd
from IPython.display import HTML, display
import clickhouse_connect
from dotenv import load_dotenv

load_dotenv()

client = clickhouse_connect.get_client(
    host=os.environ["CLICKHOUSE_HOST"],
    port=int(os.environ["CLICKHOUSE_PORT"]),
    user=os.environ["CLICKHOUSE_USER"],
    password=os.environ["CLICKHOUSE_PASSWORD"],
    database=os.environ["CLICKHOUSE_DATABASE"],
)

## List all 10-K and 10K filings for a given ticker

for this example we will list the latest 10 filings for the ticker `AMZN`.

### The query

In [4]:
tickers = {"tickers": "AMZN"}

query_filings_list = """
    SELECT
        DISTINCT r.source_report_identifier as accession_number,
        r.properties ['document_type'] as document_type,
        formatDateTime(r.reporting_period_end_date, '%Y-%m-%d') as period_date
    FROM EdgarFilingsBeta.reports r
        JOIN entities e ON e.entity_id = r.entity_id
    WHERE e.entity_ticker in ({tickers: String})
    AND r.properties ['document_type'] IN ('10-K', '10-Q')
    ORDER BY r.reporting_period_end_date DESC
    LIMIT 10
    """

### Query breakdown

The SQL query is selecting data from a table of reports and entities. Here's a breakdown of what it does:

- The `tickers` contains the parameters to be passed to the query.

- `SELECT DISTINCT r.source_report_identifier as accession_number, r.properties ['document_type'] as document_type, formatDateTime(r.reporting_period_end_date, '%Y-%m-%d') as period_date`: This part of the query is selecting three columns from the result. It's formatting the reporting_period_end_date to a specific format (YYYY-MM-DD) and renaming it to period_date. It's also renaming source_report_identifier to accession_number and properties ['document_type'] to document_type. The DISTINCT keyword ensures that each row in the result set is unique.

- `FROM EdgarFilingsBeta.reports r JOIN entities e ON e.entity_id = r.entity_id`: This part of the query specifies that the data should be fetched from the reports table in the EdgarFilingsBeta schema and the entities table. It's joining these two tables on the entity_id column.

- `WHERE e.entity_ticker in ({tickers: String}) AND r.properties ['document_type'] IN ('10-K', '10-Q')`: This is the filter condition for the query. It's selecting rows where entity_ticker is in a list of tickers and properties ['document_type'] is either '10-K' or '10-Q'.

- `ORDER BY r.reporting_period_end_date DESC`: This part of the query is ordering the results by reporting_period_end_date in descending order.

- `LIMIT 10`: This part of the query limits the result set to the first 10 rows.

### Response

Now we fetch the results from the database, and display the response.

In [5]:
result_filings_list = client.query(query_filings_list, parameters=tickers)
# Convert the result to a pandas DataFrame
df = pd.DataFrame(
    result_filings_list.result_rows, columns=result_filings_list.column_names
)

# Display the DataFrame
display(HTML(df.to_html(notebook=True)))

,accession_number,document_type,period_date
0,0001018724-24-000008,10-K,2023-12-31
1,0001018724-23-000018,10-Q,2023-09-30
2,0001018724-23-000012,10-Q,2023-06-30
3,0001018724-23-000008,10-Q,2023-03-31
4,0001018724-23-000004,10-K,2022-12-31
5,0001018724-22-000023,10-Q,2022-09-30
6,0001018724-22-000019,10-Q,2022-06-30
7,0001018724-22-000013,10-Q,2022-03-31
8,0001018724-22-000005,10-K,2021-12-31
9,0001018724-21-000028,10-Q,2021-09-30


## Display a single financial statement for a given filing

For this example, we will list the balance sheet for the report with accession number `0001018724-24-000008`. This is the `2023-12-31` `10-K` report for Amazon.com, Inc. (`AMZN`) mentioned in the previous example.

Note the ordering statements. The `network_role_description`, `relationship_tree_sequence`, and `relationship_tree_depth` fields in the `financials` table will help you build a tree-like structure of the filing.

### The query

In [6]:
accession = {"accession_number": df["accession_number"][0]}

query_filing = """
        SELECT
            formatDateTime(f.report_period_end_date, '%Y-%m-%d') as period_end,
            f.relationship_tree_sequence as sequence,
            f.relationship_tree_depth as depth,
            f.period_fiscal_year as year,
            f.period_fiscal_period as period,
            f.network_role_description as statement,
            f.relationship_target_label as label,
            f.fact_value as value
        FROM financials f
        WHERE f.report_accession IN ({accession_number: String})
            AND f.network_role_description LIKE '0000007%'
        ORDER BY f.network_role_description ASC,
            f.relationship_tree_sequence ASC,
            f.relationship_tree_depth ASC
    """


### Query breakdown

- The `accession` contains the first accession number fetched ffrom the previous query.

- `SELECT formatDateTime(f.report_period_end_date, '%Y-%m-%d') as period_end, f.relationship_tree_sequence as sequence, f.relationship_tree_depth as depth, f.period_fiscal_year as year, f.period_fiscal_period as period, f.network_role_description as statement, f.relationship_target_label as label, f.fact_value as value`: This part of the query is selecting eight columns from the result. It's formatting the report_period_end_date to a specific format (YYYY-MM-DD) and renaming it to period_end. It's also renaming several other columns for clarity or brevity.

- `FROM financials f`: This part of the query specifies that the data should be fetched from the financials table. The f after financials is an alias for the table name, allowing it to be referred to more succinctly in the rest of the query.

- `WHERE f.report_accession IN ({accession_number: String}) AND f.network_role_description LIKE '0000007%'`: This is the filter condition for the query. It's selecting rows where report_accession is in a list of accession numbers and network_role_description starts with '0000007'. The '0000007' prefix came from querying this accession number in the financials table and checking what statements where available. 

- `ORDER BY f.network_role_description ASC, f.relationship_tree_sequence ASC, f.relationship_tree_depth ASC`: This part of the query is ordering the results first by network_role_description in ascending order, then by relationship_tree_sequence in ascending order, and finally by relationship_tree_depth in ascending order. This ordering will help you build a tree-like structure of the filing.

### Response

Finally, we fetch the results from the database and display them.


In [7]:

result_filing = client.query(query_filing, parameters=accession)

# Convert the result to a pandas DataFrame
df = pd.DataFrame(result_filing.result_rows, columns=result_filing.column_names)

# Display the DataFrame
display(HTML(df.to_html(notebook=True)))

,period_end,sequence,depth,year,period,statement,label,value
0,2023-12-31,1,1,NaN,None,0000007 - Statement - Consolidated Balance Sheets,ASSETS,None
1,2023-12-31,2,2,NaN,None,0000007 - Statement - Consolidated Balance Sheets,Current assets:,None
2,2023-12-31,3,3,2023.0,Y,0000007 - Statement - Consolidated Balance Sheets,Cash and cash equivalents,73387000000
3,2023-12-31,4,3,2023.0,Y,0000007 - Statement - Consolidated Balance Sheets,Marketable securities,13393000000
4,2023-12-31,5,3,2023.0,Y,0000007 - Statement - Consolidated Balance Sheets,Inventories,33318000000
5,2023-12-31,6,3,2023.0,Y,0000007 - Statement - Consolidated Balance Sheets,"Accounts receivable, net and other",52253000000
6,2023-12-31,7,3,2023.0,Y,0000007 - Statement - Consolidated Balance Sheets,Total current assets,172351000000
7,2023-12-31,8,2,2023.0,Y,0000007 - Statement - Consolidated Balance Sheets,"Property and equipment, net",204177000000
8,2023-12-31,9,2,2023.0,Y,0000007 - Statement - Consolidated Balance Sheets,Operating leases,72513000000
9,2023-12-31,10,2,2023.0,Y,0000007 - Statement - Consolidated Balance Sheets,Goodwill,22789000000
